In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
from scipy.special import softmax

In [2]:
DEPTH_MULTIPLIER_PREFIX = '0'
DEPTH_MULTIPLIER = 25
IMAGE_SIZE = 128
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
BATCH_SIZE = 100
NUM_EPOCHS = 20

## **Load model & get feature extraction layer**

In [3]:
model_name = 'mobilenet_v1_'+ DEPTH_MULTIPLIER_PREFIX + str(DEPTH_MULTIPLIER)+'_'+str(IMAGE_SIZE)+'_epochs_'+str(NUM_EPOCHS)
model_path = 'models\\' + model_name
model = tf.keras.models.load_model(model_path)

In [4]:
#init feature extractor layer
fel = model.layers[0]
print(fel)

## **Init paths**

In [5]:
OUTPUT_FOLDER_NAME = "outputs"
DATA_NAME = "CIFAR_10"
OOD_DATA_NAME = "svhn"

FEATURES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "features")
FEATURES_DATA_FOLDER = os.path.join(FEATURES_FOLDER, DATA_NAME)
OOD_FEATURES_DATA_FOLDER = os.path.join(FEATURES_FOLDER, OOD_DATA_NAME)

DISTS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "dists")
DISTS_DATA_FOLDER = os.path.join(DISTS_FOLDER, DATA_NAME)
OOD_DISTS_DATA_FOLDER = os.path.join(DISTS_FOLDER, OOD_DATA_NAME)

CLOSEST_CLASS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "closest_classes")
CLOSEST_CLASS_DATA_FOLDER = os.path.join(CLOSEST_CLASS_FOLDER, DATA_NAME)
OOD_CLOSEST_CLASS_DATA_FOLDER = os.path.join(CLOSEST_CLASS_FOLDER, OOD_DATA_NAME)

PROBABILITIES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "probabilities")
PROBABILITIES_DATA_FOLDER = os.path.join(PROBABILITIES_FOLDER, DATA_NAME)
OOD_PROBABILITIES_DATA_FOLDER = os.path.join(PROBABILITIES_FOLDER, OOD_DATA_NAME)

LABELS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "labels")
LABELS_DATA_FOLDER = os.path.join(LABELS_FOLDER, DATA_NAME)

MEANS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "means")
MEANS_DATA_FOLDER = os.path.join(MEANS_FOLDER, DATA_NAME)    

RADIUS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "radius")
RADIUS_DATA_FOLDER = os.path.join(RADIUS_FOLDER, DATA_NAME)

## **Load in-distribution data**

In [6]:
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
TOTAL_CLASSES = len(class_names)
print(TOTAL_CLASSES)

10


In [7]:
#load train features
CIFAR_10_train_features = [None] * 10
for i in range(TOTAL_CLASSES):
    save_name = "{0}_train_features_with_{1}_samples.npy".format(class_names[i], 5000)
    save_location = os.path.join(FEATURES_DATA_FOLDER, save_name)
    CIFAR_10_train_features[i]= np.load(save_location)
CIFAR_10_train_features = np.asarray(CIFAR_10_train_features)
print(CIFAR_10_train_features.shape)

(10, 5000, 256)


In [8]:
#load validation features
CIFAR_10_validation_features = [None] * 10
for i in range(TOTAL_CLASSES):
    save_name = "{0}_validation_features_with_{1}_samples.npy".format(class_names[i], 1000)
    save_location = os.path.join(FEATURES_DATA_FOLDER, save_name)
    CIFAR_10_validation_features[i]= np.load(save_location)
CIFAR_10_validation_features = np.asarray(CIFAR_10_validation_features)
print(CIFAR_10_validation_features.shape)

(10, 1000, 256)


In [9]:
#load class means
save_name = "CIFAR_10_class_means.npy"
save_location = os.path.join(MEANS_DATA_FOLDER, save_name)
class_means = np.load(save_location)
print(class_means.shape)

(10, 256)


In [10]:
#load class radii
save_name = "CIFAR_10_class_radii.npy"
save_location = os.path.join(RADIUS_DATA_FOLDER, save_name)
class_radii = np.load(save_location)
print(class_radii.shape)

(10,)


In [11]:
#load train distances & closest classes
CIFAR_10_train_distances = [None] * 10
CIFAR_10_train_closest_classes = [None] * 10
for i in range(TOTAL_CLASSES):
    #load distances
    save_name = "{0}_train_distances_with_{1}_samples.npy".format(class_names[i], 5000)
    save_location = os.path.join(DISTS_DATA_FOLDER, save_name)
    CIFAR_10_train_distances[i]= np.load(save_location)
    #load closest classes
    save_name = "{0}_train_closest_classes_with_{1}_samples.npy".format(class_names[i], 5000)
    save_location = os.path.join(CLOSEST_CLASS_DATA_FOLDER, save_name)
    CIFAR_10_train_closest_classes[i]= np.load(save_location)
CIFAR_10_train_distances = np.asarray(CIFAR_10_train_distances)
CIFAR_10_train_closest_classes = np.asarray(CIFAR_10_train_closest_classes)
print(CIFAR_10_train_distances.shape)
print(CIFAR_10_train_closest_classes.shape)

(10, 5000)
(10, 5000)


In [12]:
#load validation distances & closest classes
CIFAR_10_validation_distances = [None] * 10
CIFAR_10_validation_closest_classes = [None] * 10
for i in range(TOTAL_CLASSES):
    #load distances
    save_name = "{0}_validation_distances_with_{1}_samples.npy".format(class_names[i], 1000)
    save_location = os.path.join(DISTS_DATA_FOLDER, save_name)
    CIFAR_10_validation_distances[i]= np.load(save_location)
    #load closest classes
    save_name = "{0}_validation_closest_classes_with_{1}_samples.npy".format(class_names[i], 1000)
    save_location = os.path.join(CLOSEST_CLASS_DATA_FOLDER, save_name)
    CIFAR_10_validation_closest_classes[i]= np.load(save_location)
CIFAR_10_validation_distances = np.asarray(CIFAR_10_validation_distances)
CIFAR_10_validation_closest_classes = np.asarray(CIFAR_10_validation_closest_classes)
print(CIFAR_10_validation_distances.shape)
print(CIFAR_10_validation_closest_classes.shape)

(10, 1000)
(10, 1000)


## **Load out-distribution data**

In [13]:
#load svhn features
save_name = "{0}_features_with_{1}_samples.npy".format(OOD_DATA_NAME, 13068)
save_location =  os.path.join(OOD_FEATURES_DATA_FOLDER, save_name)
svhn_features = np.load(save_location)
print(svhn_features.shape)

(13068, 256)


In [14]:
#load svhn distances
save_name = "{0}_distances_with_{1}_samples.npy".format(OOD_DATA_NAME, 13068)
save_location =  os.path.join(OOD_DISTS_DATA_FOLDER, save_name)
svhn_distances = np.load(save_location)
print(svhn_distances.shape)

(13068,)


In [15]:
#load svhn closest classes
save_name = "{0}_closest_classes_with_{1}_samples.npy".format(OOD_DATA_NAME, 13068)
save_location =  os.path.join(OOD_CLOSEST_CLASS_DATA_FOLDER, save_name)
svhn_closest_classes = np.load(save_location)
print(svhn_closest_classes.shape)

(13068,)
